# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff441148a30>
├── 'a' --> tensor([[-0.1166,  1.0725, -0.0604],
│                   [ 1.3568,  1.0739,  0.7051]])
└── 'x' --> <FastTreeValue 0x7ff441148970>
    └── 'c' --> tensor([[-0.6383, -0.6225, -0.1962,  1.1172],
                        [-0.6019,  1.0020,  0.0193,  0.0558],
                        [ 1.0514, -2.1353,  0.1311, -0.5507]])

In [4]:
t.a

tensor([[-0.1166,  1.0725, -0.0604],
        [ 1.3568,  1.0739,  0.7051]])

In [5]:
%timeit t.a

76.1 ns ± 0.53 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff441148a30>
├── 'a' --> tensor([[-0.3234,  0.2013, -0.8772],
│                   [-0.4889,  0.1278,  0.6607]])
└── 'x' --> <FastTreeValue 0x7ff441148970>
    └── 'c' --> tensor([[-0.6383, -0.6225, -0.1962,  1.1172],
                        [-0.6019,  1.0020,  0.0193,  0.0558],
                        [ 1.0514, -2.1353,  0.1311, -0.5507]])

In [7]:
%timeit t.a = new_value

79.2 ns ± 0.168 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1166,  1.0725, -0.0604],
               [ 1.3568,  1.0739,  0.7051]]),
    x: Batch(
           c: tensor([[-0.6383, -0.6225, -0.1962,  1.1172],
                      [-0.6019,  1.0020,  0.0193,  0.0558],
                      [ 1.0514, -2.1353,  0.1311, -0.5507]]),
       ),
)

In [10]:
b.a

tensor([[-0.1166,  1.0725, -0.0604],
        [ 1.3568,  1.0739,  0.7051]])

In [11]:
%timeit b.a

74.7 ns ± 1.49 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1209,  1.3771, -0.5161],
               [ 0.6015, -0.3667,  0.4761]]),
    x: Batch(
           c: tensor([[-0.6383, -0.6225, -0.1962,  1.1172],
                      [-0.6019,  1.0020,  0.0193,  0.0558],
                      [ 1.0514, -2.1353,  0.1311, -0.5507]]),
       ),
)

In [13]:
%timeit b.a = new_value

617 ns ± 3.83 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

969 ns ± 24.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12 µs ± 74.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

164 µs ± 503 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

180 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff391d09af0>
├── 'a' --> tensor([[[-0.1166,  1.0725, -0.0604],
│                    [ 1.3568,  1.0739,  0.7051]],
│           
│                   [[-0.1166,  1.0725, -0.0604],
│                    [ 1.3568,  1.0739,  0.7051]],
│           
│                   [[-0.1166,  1.0725, -0.0604],
│                    [ 1.3568,  1.0739,  0.7051]],
│           
│                   [[-0.1166,  1.0725, -0.0604],
│                    [ 1.3568,  1.0739,  0.7051]],
│           
│                   [[-0.1166,  1.0725, -0.0604],
│                    [ 1.3568,  1.0739,  0.7051]],
│           
│                   [[-0.1166,  1.0725, -0.0604],
│                    [ 1.3568,  1.0739,  0.7051]],
│           
│                   [[-0.1166,  1.0725, -0.0604],
│                    [ 1.3568,  1.0739,  0.7051]],
│           
│                   [[-0.1166,  1.0725, -0.0604],
│                    [ 1.3568,  1.0739,  0.7051]]])
└── 'x' --> <FastTreeValue 0x7ff391d09f70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.7 µs ± 791 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff391d09580>
├── 'a' --> tensor([[-0.1166,  1.0725, -0.0604],
│                   [ 1.3568,  1.0739,  0.7051],
│                   [-0.1166,  1.0725, -0.0604],
│                   [ 1.3568,  1.0739,  0.7051],
│                   [-0.1166,  1.0725, -0.0604],
│                   [ 1.3568,  1.0739,  0.7051],
│                   [-0.1166,  1.0725, -0.0604],
│                   [ 1.3568,  1.0739,  0.7051],
│                   [-0.1166,  1.0725, -0.0604],
│                   [ 1.3568,  1.0739,  0.7051],
│                   [-0.1166,  1.0725, -0.0604],
│                   [ 1.3568,  1.0739,  0.7051],
│                   [-0.1166,  1.0725, -0.0604],
│                   [ 1.3568,  1.0739,  0.7051],
│                   [-0.1166,  1.0725, -0.0604],
│                   [ 1.3568,  1.0739,  0.7051]])
└── 'x' --> <FastTreeValue 0x7ff391d092b0>
    └── 'c' --> tensor([[-0.6383, -0.6225, -0.1962,  1.1172],
                        [-0.6019,  1.0020,  0.0193,  0.0558],
                 

In [23]:
%timeit t_cat(trees)

34.8 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.4 µs ± 3.97 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1166,  1.0725, -0.0604],
                [ 1.3568,  1.0739,  0.7051]],
       
               [[-0.1166,  1.0725, -0.0604],
                [ 1.3568,  1.0739,  0.7051]],
       
               [[-0.1166,  1.0725, -0.0604],
                [ 1.3568,  1.0739,  0.7051]],
       
               [[-0.1166,  1.0725, -0.0604],
                [ 1.3568,  1.0739,  0.7051]],
       
               [[-0.1166,  1.0725, -0.0604],
                [ 1.3568,  1.0739,  0.7051]],
       
               [[-0.1166,  1.0725, -0.0604],
                [ 1.3568,  1.0739,  0.7051]],
       
               [[-0.1166,  1.0725, -0.0604],
                [ 1.3568,  1.0739,  0.7051]],
       
               [[-0.1166,  1.0725, -0.0604],
                [ 1.3568,  1.0739,  0.7051]]]),
    x: Batch(
           c: tensor([[[-0.6383, -0.6225, -0.1962,  1.1172],
                       [-0.6019,  1.0020,  0.0193,  0.0558],
                       [ 1.0514, -2.1353,  0.1311, -0.5507]],
         

In [26]:
%timeit Batch.stack(batches)

91 µs ± 307 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1166,  1.0725, -0.0604],
               [ 1.3568,  1.0739,  0.7051],
               [-0.1166,  1.0725, -0.0604],
               [ 1.3568,  1.0739,  0.7051],
               [-0.1166,  1.0725, -0.0604],
               [ 1.3568,  1.0739,  0.7051],
               [-0.1166,  1.0725, -0.0604],
               [ 1.3568,  1.0739,  0.7051],
               [-0.1166,  1.0725, -0.0604],
               [ 1.3568,  1.0739,  0.7051],
               [-0.1166,  1.0725, -0.0604],
               [ 1.3568,  1.0739,  0.7051],
               [-0.1166,  1.0725, -0.0604],
               [ 1.3568,  1.0739,  0.7051],
               [-0.1166,  1.0725, -0.0604],
               [ 1.3568,  1.0739,  0.7051]]),
    x: Batch(
           c: tensor([[-0.6383, -0.6225, -0.1962,  1.1172],
                      [-0.6019,  1.0020,  0.0193,  0.0558],
                      [ 1.0514, -2.1353,  0.1311, -0.5507],
                      [-0.6383, -0.6225, -0.1962,  1.1172],
                      [-0.6019,  

In [28]:
%timeit Batch.cat(batches)

166 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

416 µs ± 28.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
